In [6]:
import numpy as np
import pandas as pd
import json
import requests
import sys
import io

In [8]:
ua="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36" 
headers={"User-Agent":ua} #environment setting

In [9]:
reviews=[]
scores=[]
scoresd={}
def get_review_content(baseurl,start,end):
    
    for i in range(start,end,10):
        url=baseurl + "&start={}".format(i)
        resp = requests.get(url,headers=headers)
        jsnStr=json.loads(resp.text)
        if(len(jsnStr['reviews'])==0):
            print("no more comments, stopping")
            break
        for review in jsnStr['reviews']:
            text=review['comment']['text']
            score=review["rating"]
            #print(json.dumps(review,indent=2))
            text=text.replace("&#39;","_").replace("<br>","")
            reviews.append(text)
            scores.append(score)
            if(score not in scoresd):
                scoresd[score]=0
            scoresd[score]+=1
    return 

In [14]:
burl="https://www.yelp.com/biz/FH978pIP1TLRuPAH-MbWIQ/review_feed?rl=en&q=&sort_by=rating_asc"
burl2="https://www.yelp.com/biz/EGS6y6WsPkNs8PZ2X6bHOA/review_feed?rl=en&q=&sort_by=relevance_desc"
get_review_content(burl,0,100)

In [15]:
# # take a look on example data
example_index=0
print("review: {}".format(reviews[example_index]))
print("score: {}".format(scores[example_index]))

print(scoresd) # see how the score distributed

review: Great!  I love this place.  I found it by chance last fall while in NYC on a street photography trip.  Very friendly staff, tasty food at reasonable prices, good beers selection too.   I went back 3 times that week and would have gone again had my visit not come to an end.  There_s a beer garden-like court yard area and when the weather is good, like when I was there, they have all the doors open so a nice breeze was passing through the main eating area; sort of felt like a pleasant little greenhouse.  I_m no NYC expert but this place felt like a little oasis.
score: 4
{4: 104, 3: 633, 2: 286, 5: 22, 1: 130}


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = CountVectorizer(max_df=0.8,min_df=0.15)
vectorizer = TfidfVectorizer(max_df=0.8,min_df=0.15)

vectorizer.fit(reviews)
X=vectorizer.transform(reviews)#result in sparse matrix

In [44]:
y=[] # if score less or equal to 3, we will assign it to 0 which mean bad review,otherwise we will assign it to 1 which means good review
for i in range(len(scores)):
    if(scores[i]<=3):
        y.append(0)

    else:
        y.append(1)

In [45]:
print(X.todense()[0])
print('----')
print(X[0])
print(X.shape)
print(reviews[0])
print(len(reviews))
print(len(reviews[0]))
print(vectorizer.get_feature_names())
print(len(y))

[[0.         0.14401133 0.         0.16439168 0.         0.
  0.         0.12111703 0.1675239  0.         0.         0.
  0.         0.08837959 0.15584997 0.         0.         0.
  0.         0.161147   0.         0.         0.         0.
  0.         0.         0.         0.092997   0.         0.
  0.         0.         0.22869726 0.         0.161147   0.11673641
  0.22771503 0.         0.         0.08935794 0.08829876 0.09153252
  0.         0.         0.52635477 0.         0.         0.
  0.11571973 0.16301891 0.10311131 0.08678275 0.10858639 0.
  0.         0.         0.         0.         0.         0.
  0.         0.21018929 0.         0.         0.         0.
  0.11989111 0.         0.         0.         0.         0.10813895
  0.         0.12538904 0.10869876 0.20766921 0.         0.07544043
  0.15831568 0.         0.         0.14147135 0.         0.17419765
  0.         0.         0.         0.28371074 0.         0.
  0.15933027 0.         0.        ]]
----
  (0, 90)	0.159330

In [46]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X.todense(),y,test_size=0.21,random_state=42) #split the data to train set and test set
print(X_train.shape)
print(X_test.shape)
print(y_test)

(928, 93)
(247, 93)
[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [47]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train, y_train)
print("training done.")

training done.


In [48]:
y_pred = model.predict(X_test)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [49]:
from sklearn.metrics import classification_report

r = classification_report(y_test,y_pred)
print(r)

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       221
           1       0.67      0.08      0.14        26

    accuracy                           0.90       247
   macro avg       0.78      0.54      0.54       247
weighted avg       0.88      0.90      0.86       247



In [50]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler


#regularization
sc = StandardScaler()

xtrain = sc.fit_transform(X_train)
xtest = sc.transform(X_test)

In [28]:
model = svm.SVC()
model.fit(xtrain, y_train)
y_pred = model.predict(xtest)
print(y_pred)
r = classification_report(y_test,y_pred)
print(r)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94       221
           1       0.00      0.00      0.00        26

    accuracy                           0.89       247
   macro avg       0.45      0.50      0.47       247
weighted avg       0.80      0.89      0.85       247



c:\users\yaozl\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\yaozl\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\yaozl\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [51]:
from sklearn import tree
tr = tree.DecisionTreeClassifier()
tr.fit(X_train, y_train)
print("training done.")

training done.


In [52]:
tree_predict = tr.predict(X_test)
print(tree_predict)
rc = classification_report(y_test,tree_predict)
print(rc)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       221
           1       0.26      0.19      0.22        26

    accuracy                           0.86       247
   macro avg       0.59      0.56      0.57       247
weighted avg       0.84      0.86      0.85       247



In [53]:
from sklearn.ensemble import GradientBoostingClassifier
gdb = GradientBoostingClassifier()
gdb.fit(X_train, y_train)
print("training done.")

training done.


In [54]:
gdb_pred = gdb.predict(X_test)
gdb_report = classification_report(y_test,gdb_pred)
print(gdb_report)

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       221
           1       0.62      0.19      0.29        26

    accuracy                           0.90       247
   macro avg       0.77      0.59      0.62       247
weighted avg       0.88      0.90      0.88       247



In [59]:
import lightgbm as lgb
rng = lgb.LGBMClassifier()
rng.fit(X_train, y_train)
print('training done')

training done


In [60]:
rng_pred = rng.predict(X_test)
rng_report = classification_report(y_test,rng_pred)
print(rng_report)

              precision    recall  f1-score   support

           0       0.91      0.98      0.95       221
           1       0.56      0.19      0.29        26

    accuracy                           0.90       247
   macro avg       0.73      0.59      0.62       247
weighted avg       0.87      0.90      0.88       247

